# Generate Audio for Preset Categories

This notebook generates audio for three example presets representing the identified categories:
1. **Perfect:** `example_perfect.vital` (Init/Basic Shapes, No Sample) - Should sound correct.
2. **Good:** `example_good.vital` (Init/Basic Shapes + White Noise) - Should sound correct but cleaner (missing noise).
3. **Unusable:** `example_unusable.vital` (Custom Wavetables/Samples) - Should sound broken or silent.

In [ ]:
import os
import sys
import numpy as np
import soundfile as sf
import mido
from pedalboard import load_plugin
import IPython.display as ipd

# Add project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

from src.data.vst.core import load_preset

# Paths
PLUGIN_PATH = "../../plugins/Vital.vst3"
OUTPUT_DIR = "generated_examples"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load Plugin once
plugin = load_plugin(PLUGIN_PATH)
print("Plugin loaded.")

In [ ]:
def render_preset(preset_name, note_pitch=60, duration=4.0, overrides=None, disable_fx=False):
    preset_path = f"{preset_name}.vital"
    print(f"Loading {preset_path}...")
    
    # Load Preset
    load_preset(plugin, preset_path)
    
    # Apply Overrides
    if overrides:
        for k, v in overrides.items():
            if k in plugin.parameters:
                plugin.parameters[k].raw_value = v
                print(f"Override: Set {k} to {v}")
                
    # Disable FX if requested (to debug artifacts)
    if disable_fx:
        fx_switches = [
            "reverb_switch", "delay_switch", "compressor_switch", 
            "chorus_switch", "flanger_switch", "phaser_switch", "distortion_switch"
        ]
        for sw in fx_switches:
            if sw in plugin.parameters:
                plugin.parameters[sw].raw_value = 0.0
        print("FX Disabled.")
    
    # Flush buffer (Instrument signature)
    # Render 0.5s of silence to fully flush tails/buffers
    plugin.process([], 0.5, 44100, num_channels=2, reset=True)
    
    # Render Note
    sample_rate = 44100
    note_start = 0.5
    note_end = duration - 0.5
    
    note_on = mido.Message("note_on", note=note_pitch, velocity=100).bytes()
    note_off = mido.Message("note_off", note=note_pitch, velocity=100).bytes()
    
    midi_messages = [
        (note_on, note_start),
        (note_off, note_end)
    ]
    
    audio = plugin.process(
        midi_messages,
        duration,
        sample_rate,
        num_channels=2,
        buffer_size=2048,
        reset=True
    )
    
    # Save
    output_path = os.path.join(OUTPUT_DIR, f"{preset_name}.wav")
    sf.write(output_path, audio.T, sample_rate)
    print(f"Saved to {output_path}")
    
    return audio, sample_rate

### 1. Perfect Candidate
Should sound correct (Init/Basic oscillators only).

In [ ]:
# Disable FX to check if 'reversed echo' artifact disappears
audio, sr = render_preset("example_perfect", disable_fx=True)
ipd.Audio(audio, rate=sr)

### 2. Good Candidate
Should sound correct but might lack noise texture (Init/Basic + White Noise).
**Fix:** We enable Filter 1 explicitly because it was routed but disabled (causing silence).

In [ ]:
# Fix Silence: Enable Filter 1 (since Mix is 100%)
overrides = {
    "filter_1_switch": 1.0
}
audio, sr = render_preset("example_good", overrides=overrides, disable_fx=True)
ipd.Audio(audio, rate=sr)

### 3. Unusable Candidate
Should sound broken, silent, or like a raw Saw wave (Custom Wavetables/Samples).

In [ ]:
audio, sr = render_preset("example_unusable", disable_fx=True)
ipd.Audio(audio, rate=sr)